In [ ]:
'''
Schritt 4
Die in Schritt 3 zusammengefügte Infra (OBS und OSM) wird klassifiziert
'''

In [1]:
import json 
import pandas as pd
import geopandas as gpd
import os

from datetime import date
import glob                             #verwendung von 'wildacards' in Dateinamen
from tqdm import tqdm                   #fuer Darstellen eines Fortschrittsbalkens
import matplotlib.pyplot as plt
import numpy as np

import re       #Regex

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

print('import finished')

import finished


In [2]:
path_OBS_OSM = '../OSM-OBS_merged'  #Ort an dem Datei mit zusammengeführten OSM- und OBS-Infos (aus _1-2_OSM_OBS-merge) liegt
path_OBS_OSM = path_OBS_OSM + '/*0315.gpkg'

#liest alle obs-Dateien ein und packt sie in eine liste. 
files_OBS_OSM = glob.glob(path_OBS_OSM)

#wählt die Datei mit dem aktuellsten Bearbeitungsdatum
max(files_OBS_OSM, key=os.path.getctime)

'../OSM-OBS_merged/OSM-OBS_merge230315.gpkg'

In [3]:
gdf_merged=gpd.read_file(max(files_OBS_OSM, key=os.path.getctime)) #klassifizierte und mit OSM gemergte OBS-Messwerte
gdf_merged['Radinfra'] = "" #gdf_merged['Radinfra'].astype('string') #Datentyp String für Radinfra Spalte. Vorbereitung für das Auslesen.
gdf_merged.head()

,distance_overtaker,distance_stationary,direction,way_id,course,speed,time,portal,osm_id,name,highway,ref,maxspeed,zone_traffic,lanes,lane_markings,lit,surface,bicycle,cycleway,cycleway_both,cycleway_left,cycleway_right,width,tracktype,smoothness,oneway,cycleway_lane,cycleway_both_lane,cycleway_right_lane,cycleway_left_lane,parking_lane_both,parking_lane_left,parking_lane_right,cycleway_separation_left,cycleway_separation_right,separation_left,separation_right,cycleway_bicycle,cycleway_both_bicycle,...,parking_left,parking_right,cycleway_right_separation_right,cycleway_left_separation_right,cycleway_both_separation_right,parking_both_orientation,parking_left_orientation,parking_right_orientation,cycleway_right_separation_left,cycleway_left_separation_left,cycleway_both_separation_left,cycleway_right_separation_both,cycleway_left_separation_both,cycleway_both_separation_both,cycleway_right_marking_both,cycleway_left_marking_both,cycleway_both_marking_left,cycleway_both_marking_right,cycleway_both_traffic_sign,cycleway_both_marking_both,cycleway_right_buffer_right,cycleway_left_buffer_right,cycleway_both_buffer_right,cycleway_right_buffer_left,cycleway_left_buffer_left,cycleway_both_buffer_left,cycleway_right_buffer_both,cycleway_left_buffer_both,cycleway_both_buffer_both,cycleway_marking_both,cycleway_marking_left,cycleway_marking,cycleway_right_marking,cycleway_left_marking,cycleway_traffic_sign,parking_orientation,other_tags,geometry_y,geometry,Radinfra
0,NaN,2.64,1,141561211,2.956398,3.083333,2022-08-11 09:38:31+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32561 51.76605),
1,1.67,1.70,1,141561211,2.895165,3.750000,2022-08-11 09:38:32+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32553 51.76606),
2,0.85,2.79,1,141561211,2.849412,4.055556,2022-08-11 09:38:34+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32541 51.76607),
3,1.26,NaN,1,141561211,2.900393,4.250000,2022-08-11 09:38:36+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32527 51.76609),
4,1.96,2.12,1,141561211,2.910295,4.305556,2022-08-11 09:38:37+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,aspha

In [4]:
#Herausfiltern von Messungen (links) unter 5cm
gdf_filter=gdf_merged[~(gdf_merged.distance_overtaker<0.05)]
#gdf_filter=gdf_merged[~(gdf_merged.distance_stationary<0.05)]
#Herausfiltern von Messungen mit Geschw. Fahrrad > 12m/s (>43km/h)
#gdf_filter=gdf_merge[~(gdf_merge.speed>12)]
len(gdf_filter)

19711

In [6]:
gdf_filter.head()

,distance_overtaker,distance_stationary,direction,way_id,course,speed,time,portal,osm_id,name,highway,ref,maxspeed,zone_traffic,lanes,lane_markings,lit,surface,bicycle,cycleway,cycleway_both,cycleway_left,cycleway_right,width,tracktype,smoothness,oneway,cycleway_lane,cycleway_both_lane,cycleway_right_lane,cycleway_left_lane,parking_lane_both,parking_lane_left,parking_lane_right,cycleway_separation_left,cycleway_separation_right,separation_left,separation_right,cycleway_bicycle,cycleway_both_bicycle,...,parking_left,parking_right,cycleway_right_separation_right,cycleway_left_separation_right,cycleway_both_separation_right,parking_both_orientation,parking_left_orientation,parking_right_orientation,cycleway_right_separation_left,cycleway_left_separation_left,cycleway_both_separation_left,cycleway_right_separation_both,cycleway_left_separation_both,cycleway_both_separation_both,cycleway_right_marking_both,cycleway_left_marking_both,cycleway_both_marking_left,cycleway_both_marking_right,cycleway_both_traffic_sign,cycleway_both_marking_both,cycleway_right_buffer_right,cycleway_left_buffer_right,cycleway_both_buffer_right,cycleway_right_buffer_left,cycleway_left_buffer_left,cycleway_both_buffer_left,cycleway_right_buffer_both,cycleway_left_buffer_both,cycleway_both_buffer_both,cycleway_marking_both,cycleway_marking_left,cycleway_marking,cycleway_right_marking,cycleway_left_marking,cycleway_traffic_sign,parking_orientation,other_tags,geometry_y,geometry,Radinfra
0,NaN,2.64,1,141561211,2.956398,3.083333,2022-08-11 09:38:31+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,2,no,yes,asphalt,None,None,lane,None,None,8,None,intermediate,None,None,advisory,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,kerb,None,None,None,None,None,None,None,None,None,None,None,dashed_line,None,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32561 51.76605),
1,1.67,1.70,1,141561211,2.895165,3.750000,2022-08-11 09:38:32+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,2,no,yes,asphalt,None,None,lane,None,None,8,None,intermediate,None,None,advisory,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,kerb,None,None,None,None,None,None,None,None,None,None,None,dashed_line,None,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32553 51.76606),
2,0.85,2.79,1,141561211,2.849412,4.055556,2022-08-11 09:38:34+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,2,no,yes,asphalt,None,None,lane,None,None,8,None,intermediate,None,None,advisory,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,kerb,None,None,None,None,None,None,None,None,None,None,None,dashed_line,None,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32541 51.76607),
3,1.26,NaN,1,141561211,2.900393,4.250000,2022-08-11 09:38:36+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,2,no,yes,asphalt,None,None,lane,None,None,8,None,intermediate,None,None,advisory,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,kerb,None,None,None,None,None,None,None,None,None,None,None,dashed_line,None,none,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32527 51.76609),
4,1.96,2.12,1,141561211,2.910295,4.305556,2022-08-11 09:38:37+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None

In [5]:
gdf_sort=gdf_filter
#gdf_sort = gdf_sort[gdf_sort['osm_id'] == 373996566]

#Ausgelagerte Funktion. Checkt, um welche Art von ???streifen es sich handelt.
def check_pflicht(reihe, bisher):
    pflicht = "...streifen,"
    pflicht = str(pflicht)
    #wenn ein way eine doppelte OSM-Definition für den Streifen hat, wird nur eine ausgewertet.
    #welche ausgewertet wird hängt an der Reihenfolge, in der in dieser Funktion die Reihen geprüft werden.
    if 'Radfahrstreifen' in bisher or 'Schutzstreifen' in bisher or 'Seitenstreifen' in bisher or 'PBL' in bisher:
        return(bisher)
        pass
    else:
        #Prüfung auf Kriterien eines Seitenstreifens (durchgezogene Linie und KEIN Radwegschild)
        if reihe["cycleway_both_marking_left"]=='solid_line' and reihe["cycleway_both_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_both_marking_both"]=='solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking_both"]=='solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking_left"]=='solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking"]=='solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_both_marking_left"]=='double_solid_line' and reihe["cycleway_both_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_both_marking_both"]=='double_solid_line' and reihe["cycleway_both_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking_both"]=='double_solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking_left"]=='double_solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        elif reihe["cycleway_marking"]=='double_solid_line' and reihe["cycleway_traffic_sign"]=='none':
            pflicht='Seitenstreifen, '
        #Seitenweise überprüfung
        if reihe['direction']==1:
            if reihe["cycleway_right_marking_left"]=='solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_right_marking_left"]=='double_solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_right_marking_both"]=='solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_right_marking_both"]=='double_solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_right_marking"]=='solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_right_marking"]=='double_solid_line' and reihe["cycleway_right_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
        if reihe['direction']==-1:
            if reihe["cycleway_left_marking_left"]=='solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_left_marking_left"]=='double_solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_left_marking_both"]=='solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_left_marking_both"]=='double_solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_left_marking"]=='solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
            elif reihe["cycleway_left_marking"]=='double_solid_line' and reihe["cycleway_left_traffic_sign"]=='none':
                pflicht='Seitenstreifen, '
        
        #Für Radfahr- und Schutzstreifen nun als erstes die richtungsspezifischen Angaben überprüfen. 
        #Annahme: Detailliertere Beschreibung ist auch korrekter.
        if reihe['direction']==1:
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
            #Falls ein geschützter Radfahrstreifen nicht als separater Way angelegt wurde.
                try:     #wenn kein Wert eingetragen kommt Fehler: kann NoneType nicht zu String konvertieren.
                    if re.search(r'|'.join(liste_PBL), reihe['cycleway_right_separation_left']):
                        pflicht ='PBL,'
                except BaseException as e:
                    pass
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
                if reihe['cycleway_right_lane']=='exclusive':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_right_lane']=='designated':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_right_lane']=='advisory':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_right_lane']=='yes':
                    pflicht = "Schutzstreifen,"
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
                if reihe['cycleway_right_bicycle']=='advisory':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_right_bicycle']=='yes':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_right_bicycle']=='designated':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_right_bicycle']=='exclusive':
                    pflicht = "Radfahrstreifen,"
        if reihe['direction']==-1:
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
                try:
                    if re.search(r'|'.join(liste_PBL), reihe['cycleway_left_separation_left']):
                        pflicht ='PBL,'
                except BaseException as e:
                    pass            
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
                if reihe['cycleway_left_lane']=='exclusive':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_left_lane']=='designated':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_left_lane']=='advisory':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_left_lane']=='yes':
                    pflicht = "Schutzstreifen,"
            if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
                pass
            else:
                if reihe['cycleway_left_bicycle']=='advisory':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_left_bicycle']=='yes':
                    pflicht = "Schutzstreifen,"
                elif reihe['cycleway_left_bicycle']=='designated':
                    pflicht = "Radfahrstreifen,"
                elif reihe['cycleway_left_bicycle']=='exclusive':
                    pflicht = "Radfahrstreifen,"                     
        #Auswertung RichtungsUNspezifischer Angaben
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            try:
                if re.search(r'|'.join(liste_PBL), reihe['cycleway_separation_left']):
                    pflicht ='PBL,'
            except BaseException as e:
                pass
            try:
                if re.search(r'|'.join(liste_PBL), reihe['cycleway_both_separation_left']):
                    pflicht ='PBL,'
            except BaseException as e:
                pass 
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            if reihe['cycleway_both_lane']=='exclusive':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_both_lane']=='designated':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_both_lane']=='advisory':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_both_lane']=='yes':
                pflicht = "Schutzstreifen,"
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            if reihe['cycleway_lane']=='exclusive':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_lane']=='designated':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_lane']=='advisory':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_lane']=='yes':
                pflicht = "Schutzstreifen,"
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            if reihe['cycleway_bicycle']=='advisory':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_bicycle']=='yes':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_bicycle']=='designated':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_bicycle']=='exclusive':
                pflicht = "Radfahrstreifen,"
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            if reihe['cycleway_both_bicycle']=='advisory':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_both_bicycle']=='yes':
                pflicht = "Schutzstreifen,"
            elif reihe['cycleway_both_bicycle']=='designated':
                pflicht = "Radfahrstreifen,"
            elif reihe['cycleway_both_bicycle']=='exclusive':
                pflicht = "Radfahrstreifen,"
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            #sollte eig. nicht vorkommen. Hieße, dass ein Fahrradstreifen als eigenständiger Weg getagged wurde.
            if reihe['bicycle']=='designated':
                pflicht = "Radfahrstreifen,"
            elif reihe['bicycle']=='exclusive':
                pflicht = "Radfahrstreifen,"
            elif reihe['bicycle']=='advisory':
                pflicht = "Schutzstreifen,"
            elif reihe['bicycle']=='yes':
                pflicht = "Schutzstreifen,"
        if 'Radfahrstreifen' in pflicht or 'Schutzstreifen' in pflicht or 'Seitenstreifen' in pflicht or 'PBL' in pflicht:
            pass
        else:
            if reihe['highway']=='use_sidepath':  #hä? was für ein fall soll das sein?
                pflicht = "Radfahrstreifen,"
        #print(pflicht,reihe['way_id'])
        return (str(pflicht))
        
#---------------------------------------------
#Einfügen von leeren Spalten, um sie im folgenden mit Infos zu füllen.
gdf_sort['Radwegbreite']=np.nan
gdf_sort['PufferRechts']=np.nan 
gdf_sort['PufferLinks']=np.nan 

gdf_sort['GG-Radstreifen']=np.nan
gdf_sort['GG-Radwegbreite']=np.nan
gdf_sort['GG-PufferRechts']=np.nan 
gdf_sort['GG-PufferLinks']=np.nan 

gdf_sort['Fahrstreifenbreite']=np.nan
gdf_sort['FSBundRad']=np.nan 
gdf_sort['Kernfahrbahn']=np.nan 
gdf_sort['Fahrbahn_Fliessverkehr']=np.nan 

gdf_sort['Ortslage']=np.nan
gdf_sort['Geschwindigkeit_Numerisch']=np.nan

#Liste mit Trennelementen, die, wenn links des Radwegs, eine Protected Bike Lane (PBL) erzeugen. 
liste_PBL = ['bollard', 'planter', 'flex_post', 'parking_lane', 'bump', 'fence', 'jersey_barrier', 'separation_kerb', 'vertival_panel', 'kerb']
liste_PBL = [re.escape(m) for m in liste_PBL]  #Falls vorhanden werden Zeichen, die in Regex eine Bedeutung haben ('$', ".", uvm.), ersetzt, sodass nach Ihnen gesucht werden kann
#Liste mit Fällen, wo Überholvorgänge (vsl.) nicht im Mischverkehr stattgefunden haben 
liste_Misch = ['PBL', 'Radfahrstreifen', 'Schutzstreifen', 'Seitenstreifen', '...streifen', 'Busfahrstreifen']
liste_Misch = [re.escape(m) for m in liste_Misch]

for i,row in gdf_sort.iterrows():
    infra=''  #Allgemeine Infrastrukturmerkmale inkl. Radinfra
    pinfra='' #Parkinfra
    lage=''   #inner oder ausserorts

    #Element überspringne, falls keine OSM-ID vorhanden. Sollte nicht vorkommen.
    if pd.isnull(gdf_sort['osm_id'][i]):
        infra+='kein-OSM'
    else:
        #-----Geschwindigkeit in numerische Werte umwandeln -> spaeter einfacher auswertbar-------
        if not pd.isnull(gdf_sort['maxspeed'][i]):
            if row['maxspeed'] == 'DE:urban':                    #Regelgeschwindigkeit innerorts
                gdf_sort['Geschwindigkeit_Numerisch'][i]='50'
            elif row['maxspeed'] == 'walk':                      #Fussgaengerzone
                gdf_sort['Geschwindigkeit_Numerisch'][i]='7' 
            elif row['maxspeed'] == 'DE:living_street':          #Fussgaengerzone
                gdf_sort['Geschwindigkeit_Numerisch'][i]='7'
            elif row['maxspeed'] == 'DE:rural':                  #Regelgeschwindigkeit ausserorts
                gdf_sort['Geschwindigkeit_Numerisch'][i]='100'  
            elif row['maxspeed'] == 'DE:bicycle_road':           #Fahrradstrasse
                gdf_sort['Geschwindigkeit_Numerisch'][i]='30'
            elif row['maxspeed'] == 'none':                      #Autobahn
                gdf_sort['Geschwindigkeit_Numerisch'][i]='300'      #hoher Wert, um numerischen Wert für Auswertung zu haben
            elif row['maxspeed'] == 'DE:motorway':               #Autobahn
                gdf_sort['Geschwindigkeit_Numerisch'][i]='300'  
            else:
                gdf_sort['Geschwindigkeit_Numerisch'][i]=row['maxspeed']
        #-----Messpunkt inner- oder ausserorts?-------
            ## Einzige direkte Information dazu in key: traffic:zone zu finden. Dies aber noch wenig genutzt.
            ## Für den Rest wird daher vereinfachend davon ausgegangen, dass alle Abschnitte mit einer  
            ## Geschwindigkeitsbegrenzung von 50km/h oder weniger innerorts liegen.
        if not pd.isnull(gdf_sort['zone_traffic'][i]):
            if row['zone_traffic'] == 'DE:urban':
                lage = 'inner'
            elif row['zone_traffic'] == 'DE:rural':
                lage = 'ausser'
            elif row['zone_traffic'] == 'DE:motorway':
                lage = 'ausser'
        else:
            if not pd.isnull(gdf_sort['Geschwindigkeit_Numerisch'][i]):
                if int(gdf_sort['Geschwindigkeit_Numerisch'][i]) <= 50:
                    lage = 'inner'
                elif int(gdf_sort['Geschwindigkeit_Numerisch'][i]) > 50:
                    lage = 'ausser'
            else:
                lage = 'unbekannt'
        gdf_sort['Ortslage'][i]=lage
        
        #checken ob in oder ggrichtung des OSM Elements
        if row['direction']==1:
            infra+='1,'
            #------Radinfratyp--------------
            #---------separat getagged (z.B. PBL)------
            if row['highway']=='cycleway':
                #Checken ob PBL, sonst Radweg
                if re.search(r'|'.join(liste_PBL), row['separation_left']):  #erzeugt einen String mit allen Elementen der Liste getrennt durhc '|' (logisches Oder)
                    infra+='PBL,'   #protected bike lane
                else:    
                    infra+='Radweg,'
                if not pd.isnull(gdf_sort['width'][i]):
                    infra+='Radbreite:' + row['width'] + ','   
            #---------am Hauptway getaggte Radwege-----
            elif row['cycleway']=='track':  #für ggrichtung auch opposite_track
                infra+='Radweg,'
            elif row['cycleway_both']=='track':
                infra+='Radweg,'
            elif row['cycleway_right']=='track':
                infra+='Radweg,'
            elif row['cycleway']=='separate': 
                infra+='Radweg,'
            elif row['cycleway_both']=='separate':
                infra+='Radweg,'
            elif row['cycleway_right']=='separate':
                infra+='Radweg,'
            elif row['cycleway']=='share_busway': 
                infra+='Busfahrstr.,'
            elif row['cycleway_both']=='share_busway':
                infra+='Busfahrstr.,'
            elif row['cycleway_right']=='share_busway':
                infra+='Busfahrstr.,'
            elif row['cycleway']=='shared_lane': 
                infra+='Piktogrammkette,'
            elif row['cycleway_both']=='shared_lane':
                infra+='Piktogrammkette,'
            elif row['cycleway_right']=='shared_lane':
                infra+='Piktogrammkette,'
            elif row['bicycle']=='use_sidepath':
                infra+='Radweg,' 
            #---------Fahrradstrasse------
            if row['bicycle_road']=='yes':       
                infra+='Fahrradstr,'
            #---------???Streifen---------
            #Herausfinden der Art des Streifens in ausgelagerter Funktion (oben)
            if row['cycleway']=='lane':
                infra += check_pflicht(row, infra)
            if row['cycleway_both']=='lane':
                infra += check_pflicht(row, infra)
            if row['cycleway_right']=='lane':  #ggrichtung
                infra += check_pflicht(row, infra)
            #Herausfinden, ob auf Gegenseite ein ?Streifen existiert (wenn ja wird er später bei der Berechnung der Kernfahrbahn abgezogen)
            # PBL nicht berücksichtigt
            if row['cycleway']=='lane':
                gdf_sort['GG-Radstreifen'][i] = str(check_pflicht(row, infra))  #str() nur als Test, eig. sollte nicht benötigt werden.
            if row['cycleway_both']=='lane':
                gdf_sort['GG-Radstreifen'][i] = str(check_pflicht(row, infra))
            if row['cycleway_left']=='lane':  #ggrichtung
                gdf_sort['GG-Radstreifen'][i] = str(check_pflicht(row, infra))
            #Für den Rest (Straßen ohne: baulichen Radweg, RFS, SSF, SEI, ...streifen, PBL) noch "Misch" hinzufügen
            #Liste: siehe oben
            if re.search(r'|'.join(liste_Misch), infra):
                #print(infra)
                pass
            else:
                infra += 'Misch, '
                #print(infra)
            #------Verkehrszeichen------
            if not pd.isnull(gdf_sort['cycleway_traffic_sign'][i]):
                if row['cycleway_traffic_sign'] == 'none' or row['cycleway_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_traffic_sign']) + ', '
            if not pd.isnull(gdf_sort['cycleway_both_traffic_sign'][i]):
                if row['cycleway_both_traffic_sign'] == 'none' or row['cycleway_both_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_both_traffic_sign']) + ', '
            if not pd.isnull(gdf_sort['cycleway_right_traffic_sign'][i]):     #1xggrichtung
                if row['cycleway_right_traffic_sign'] == 'none' or row['cycleway_right_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_right_traffic_sign']) + ', '
            #------marking--------------
            if row['cycleway_right_marking_left']=='solid_line':    #4xggrichtung
                infra+='linkeM_SL, '
            elif row['cycleway_right_marking_left']=='double_solid_line':
                infra+='linkeM_DSL, '
            elif row['cycleway_right_marking_left']=='dashed_line':
                infra+='linkeM_DL, '
            elif row['cycleway_right_marking_left']=='barred_area':
                infra+='linkeM_BA, '
            if row['cycleway_right_marking_right']=='solid_line':   #4xggrichtung
                infra+='rechteM_SL,'
            elif row['cycleway_right_marking_right']=='double_solid_line':
                infra+='rechteM_DSL, '
            elif row['cycleway_right_marking_right']=='dashed_line':
                infra+='rechteM_DL, '
            elif row['cycleway_right_marking_right']=='barred_area':
                infra+='rechteM_BA, '
            if row['cycleway_right_marking_both']=='solid_line':     #4xggrichtung
                infra+='rechteM_SL, linkeM_SL, '
            elif row['cycleway_right_marking_both']=='double_solid_line':
                infra+='rechteM_DSL, linkeM_DSL, '
            elif row['cycleway_right_marking_both']=='dashed_line':
                infra+='rechteM_DL, linkeM_DL, '
            elif row['cycleway_right_marking_both']=='barred_area':
                infra+='rechteM_BA, linkeM_BA, '
            if row['cycleway_both_marking_left']=='solid_line':
                infra+='linkeM_SL, '
            elif row['cycleway_both_marking_left']=='double_solid_line':
                infra+='linkeM_DSL, '
            elif row['cycleway_both_marking_left']=='dashed_line':
                infra+='linkeM_DL, '
            elif row['cycleway_both_marking_left']=='barred_area':
                infra+='linkeM_BA, '
            if row['cycleway_both_marking_right']=='solid_line':
                infra+='rechteM_SL, '
            elif row['cycleway_both_marking_right']=='double_solid_line':
                infra+='rechteM_DSL, '
            elif row['cycleway_both_marking_right']=='dashed_line':
                infra+='rechteM_DL, '
            elif row['cycleway_both_marking_right']=='barred_area':
                infra+='rechteM_BA, '
            if row['cycleway_both_marking_both']=='solid_line':
                infra+='rechteM_SL, linkeM_SL, '
            elif row['cycleway_both_marking_both']=='double_solid_line':
                infra+='rechteM_DSL, linkeM_DSL, '
            elif row['cycleway_both_marking_both']=='dashed_line':
                infra+='rechteM_DL, linkeM_DL, '
            elif row['cycleway_both_marking_both']=='barred_area':
                infra+='rechteM_BA, linkeM_BA, '
            #------Separation-----------
            if row['cycleway_right_separation_left']=='kerb':    #6xggrichtung
                infra+='linkeSep_Bord, '
            elif row['cycleway_right_separation_left']=='greenery':
                infra+='linkeSep_Gruen, '
            if row['cycleway_right_separation_right']=='kerb':
                infra+='rechteSep_Bord, '
            elif row['cycleway_right_separation_right']=='greenery':
                infra+='rechteSep_Gruen, '
            if row['cycleway_right_separation_both']=='kerb':
                infra+='linkeSep_Bord, rechteSep_Bord, '
            elif row['cycleway_right_separation_both']=='greenery':
                infra+='linkeSep_Gruen, rechteSep_Gruen, '
            if row['cycleway_both_separation_left']=='kerb':
                infra+='linkeSep_Bord, '
            elif row['cycleway_both_separation_left']=='greenery':
                infra+='linkeSep_Gruen, '
            if row['cycleway_both_separation_right']=='kerb':
                infra+='rechteSep_Bord, '
            elif row['cycleway_both_separation_right']=='greenery':
                infra+='rechteSep_Gruen, '
            if row['cycleway_both_separation_both']=='kerb':
                infra+='linkeSep_Bord, rechteSep_Bord, '
            elif row['cycleway_both_separation_both']=='greenery':
                infra+='linkeSep_Gruen, rechteSep_Gruen, '
            #------Buffer---------------
            if not pd.isnull(gdf_sort['cycleway_right_buffer_right'][i]):
                infra+='rechterPuffer:' + row['cycleway_right_buffer_right'] + ',' #3 bzw 4x ggrichtung
                gdf_sort['PufferRechts'][i]=row['cycleway_right_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_right_buffer_left'][i]):
                infra+='linkerPuffer:' + row['cycleway_right_buffer_left'] + ','
                gdf_sort['PufferLinks'][i]=row['cycleway_right_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_right_buffer_both'][i]):
                infra+='rechterPuffer:' + row['cycleway_right_buffer_both'] + ', linkerPuffer:' + row['cycleway_right_buffer_both'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_right_buffer_both']
                gdf_sort['PufferLinks'][i]=row['cycleway_right_buffer_both']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_right'][i]):
                infra+='rechterPuffer:' + row['cycleway_both_buffer_right'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_both_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_left'][i]):
                infra+='linkerPuffer:' + row['cycleway_both_buffer_left'] + ','
                gdf_sort['PufferLinks'][i]=row['cycleway_both_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_both'][i]):
                infra+='rechterPuffer:' + row['cycleway_both_buffer_both'] + ', linkerPuffer:' + row['cycleway_both_buffer_both'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_both_buffer_both']
                gdf_sort['PufferLinks'][i]=row['cycleway_both_buffer_both']
            #------Buffer - Gegenrichtung------
            if not pd.isnull(gdf_sort['cycleway_left_buffer_right'][i]):    #3 bzw 4x ggrichtung
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_left_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_left_buffer_left'][i]):
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_left_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_left_buffer_both'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_left_buffer_both']
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_left_buffer_both']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_right'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_both_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_left'][i]):
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_both_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_both'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_both_buffer_both']
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_both_buffer_both']
            #------Radwegbreite-----------------
            if not pd.isnull(gdf_sort['cycleway_right_width'][i]):
                infra+='Radbreite:' + row['cycleway_right_width'] + ',' #ggrichtung
                gdf_sort['Radwegbreite'][i]=row['cycleway_right_width']
            elif not pd.isnull(gdf_sort['cycleway_both_width'][i]):
                infra+='Radbreite:' + row['cycleway_both_width'] + ','
                gdf_sort['Radwegbreite'][i]=row['cycleway_both_width']
            elif not pd.isnull(gdf_sort['cycleway_width'][i]):
                infra+='Radbreite:' + row['cycleway_width'] + ','
                gdf_sort['Radwegbreite'][i]=row['cycleway_width']
            #------Radwegbreite- Gegenrichtung--
            if not pd.isnull(gdf_sort['cycleway_left_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_left_width']
            elif not pd.isnull(gdf_sort['cycleway_both_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_both_width']
            elif not pd.isnull(gdf_sort['cycleway_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_width']
            #------Parken-----------------------
            if row['parking_lane_both']=='perpendicular':
                infra+='senkrechtparken,'
            elif row['parking_lane_both']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_lane_both']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_lane_both']=='seperat':
                infra+='parken_extra,'
            elif row['parking_lane_both']=='marked':
                infra+='parken_extra,'
            elif row['parking_lane_both']=='no' or row['parking_lane_both']=='none':
                infra+='kein_P,'
            if row['parking_lane_right']=='perpendicular': #5xggrichtung
                infra+='senkrechtparken,'
            elif row['parking_lane_right']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_lane_right']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_lane_right']=='seperat':
                infra+='parken_extra,'
            elif row['parking_lane_right']=='marked':
                infra+='parken_extra,'
            elif row['parking_lane_right']=='no' or row['parking_lane_right']=='none':
                infra+='kein_P,'
            if row['parking']=='lane':
                infra+='Parkstrei., '
            elif row['parking']=='street_side':
                infra+='Parkbucht, '
            elif row['parking']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking']=='seperat':
                infra+='parken_extra,'
            elif row['parking']=='marked':
                infra+='parken_extra,'
            elif row['parking']=='no' or row['parking']=='none':
                infra+='kein_P,'
            if row['parking_right']=='lane': #5xggrichtung
                infra+='Parkstrei., '
            elif row['parking_right']=='street_side':
                infra+='Parkbucht, '
            elif row['parking_right']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking_right']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking_right']=='separate':
                infra+='parken_extra, '
            elif row['parking_right']=='no' or row['parking_right']=='none':
                infra+='kein_P,'
            if row['parking_both']=='lane':
                infra+='Parkstrei., '
            elif row['parking_both']=='street_side':
                infra+='Parkbucht, '
            elif row['parking_both']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking_both']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking_both']=='separate':
                infra+='parken_extra, '
            elif row['parking_both']=='no' or row['parking_both']=='none':
                infra+='kein_P,'
            if row['parking_orientation']=='perpendicular': 
                infra+='senkrechtparken,'
            elif row['parking_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_orientation']=='marked':
                infra+='parken_extra,'
            if row['parking_right_orientation']=='perpendicular': #5xggrichtung
                infra+='senkrechtparken,'
            elif row['parking_right_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_right_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_right_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_right_orientation']=='marked':
                infra+='parken_extra,'
            if row['parking_both_orientation']=='perpendicular': 
                infra+='senkrechtparken,'
            elif row['parking_both_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_both_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_both_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_both_orientation']=='marked':
                infra+='parken_extra,'
            #------Parken-Gegenrichtung-------------
            pinfra = '(P-gg: '
            if row['parking_lane_both']=='perpendicular':
                pinfra+='senkrechtparken,'
            elif row['parking_lane_both']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_lane_both']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_lane_both']=='seperat':
                pinfra+='parken_extra,'
            elif row['parking_lane_both']=='marked':
                pinfra+='parken_extra,'
            elif row['parking_lane_both']=='no' or row['parking_lane_both']=='none':
                pinfra+='kein_P,'
            if row['parking_lane_left']=='perpendicular': #5xggrichtung
                pinfra+='senkrechtparken,'
            elif row['parking_lane_left']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_lane_left']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_lane_left']=='seperat':
                pinfra+='parken_extra,'
            elif row['parking_lane_left']=='marked':
                pinfra+='parken_extra,'
            elif row['parking_lane_left']=='no' or row['parking_lane_left']=='none':
                pinfra+='kein_P,'
            if row['parking']=='lane':
                pinfra+='Parkstrei., '
            elif row['parking']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking']=='seperate':
                pinfra+='parken_aufgesetzt, '
            elif row['parking']=='marked':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking']=='no' or row['parking']=='none':
                pinfra+='kein_P,'
            if row['parking_left']=='lane': #5xggrichtung
                pinfra+='Parkstrei., '
            elif row['parking_left']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking_left']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking_left']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking_left']=='separate':
                pinfra+='parken_extra, '
            elif row['parking_left']=='no' or row['parking_left']=='none':
                pinfra+='kein_P,'
            if row['parking_both']=='lane':
                pinfra+='Parkstrei., '
            elif row['parking_both']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking_both']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking_both']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking_both']=='separate':
                pinfra+='parken_extra, '
            elif row['parking_both']=='no' or row['parking_both']=='none':
                pinfra+='kein_P,'
            if row['parking_orientation']=='perpendicular': 
                pinfra+='senkrechtparken,'
            elif row['parking_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_orientation']=='marked':
                pinfra+='parken_extra,'
            if row['parking_left_orientation']=='perpendicular': #5xggrichtung
                pinfra+='senkrechtparken,'
            elif row['parking_left_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_left_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_left_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_left_orientation']=='marked':
                pinfra+='parken_extra,'
            if row['parking_both_orientation']=='perpendicular': 
                pinfra+='senkrechtparken,'
            elif row['parking_both_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_both_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_both_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_both_orientation']=='marked':
                pinfra+='parken_extra,'
            pinfra+='), '
            if pinfra == '(P-gg: ), ':
                pinfra=''
            infra += pinfra
            #-----Straßenbreite---------------
            if not pd.isnull(gdf_sort['width'][i]):
                infra+='Gesamtbreite:' + row['width'] + ','
            #-----Einbahnstr. oder nicht?-----
            if row['oneway']=='yes':
                infra+='Einricht,'
            elif row['oneway']=='no':
                infra+='Zweiricht,' 
            if infra=='1,':
                infra+='Misch'

        if row['direction']==-1:
            infra+='-1,'
            #------Radinfratyp--------------
            #---------separat getagged (z.B. PBL)------
            if row['highway']=='cycleway':
                if re.search(r'|'.join(liste_PBL), row['separation_left']):
                    infra+='PBL,'   #protected bike lane
                else:    
                    infra+='Radweg,'
                if not pd.isnull(gdf_sort['width'][i]):
                    infra+='Radbreite:' + row['width'] + ','
            #---------am Hauptway getaggte Radwege-----
            elif row['cycleway']=='track':  
                infra+='Radweg,'
            elif row['cycleway']=='opposite_track':  #für ggrichtung auch opposite_track (Einbahnstr. mit Radinfra in ggrichtung)
                infra+='Radweg,'
            elif row['cycleway_both']=='track':
                infra+='Radweg,'
            elif row['cycleway_left']=='track':
                infra+='Radweg,'
            elif row['cycleway']=='separate':
                infra+='Radweg,'
            elif row['cycleway_both']=='separate':
                infra+='Radweg,'
            elif row['cycleway_left']=='separate':
                infra+='Radweg,'
            elif row['cycleway']=='share_busway': 
                infra+='Busfahrstr.,'
            elif row['cycleway_both']=='share_busway':
                infra+='Busfahrstr.,'
            elif row['cycleway_left']=='share_busway':
                infra+='Busfahrstr.,'
            elif row['cycleway']=='shared_lane': 
                infra+='Piktogrammkette,'
            elif row['cycleway_both']=='shared_lane':
                infra+='Piktogrammkette,'
            elif row['cycleway_left']=='shared_lane':
                infra+='Piktogrammkette,'
            elif row['bicycle']=='use_sidepath':
                infra+='Radweg,'
            #---------Fahrradstrasse------
            if row['bicycle_road']=='yes':
                infra+='Fahrradstr,'
            #---------???Streifen---------
            if row['cycleway']=='lane':
                infra += check_pflicht(row, infra)
            if row['cycleway_both']=='lane':
                infra += check_pflicht(row, infra)
            if row['cycleway_left']=='lane':  #ggrichtung
                infra += check_pflicht(row, infra)

            #Herausfinden, ob auf Gegenseite ein ?Streifen existiert (wenn ja wird er später bei der Berechnung der Kernfahrbahn abgezogen)
            #separat getaggte PBL nicht berücksichtigt
            if row['cycleway']=='lane':
                gdf_sort['GG-Radstreifen'][i] = check_pflicht(row, infra)
            if row['cycleway_both']=='lane':
                gdf_sort['GG-Radstreifen'][i] = check_pflicht(row, infra)
            if row['cycleway_right']=='lane':  #ggrichtung
                gdf_sort['GG-Radstreifen'][i] = check_pflicht(row, infra)   
            #Ermittlung Breite des Radwegs (auf eigener und Gegenseite) später
            
            #Für den Rest (Straßen ohne: baulichen Radweg, RFS, SSF, SEI, ...streifen, PBL) noch "Misch" hinzufügen
            #Liste: siehe oben
            if re.search(r'|'.join(liste_Misch), infra):
                #print(infra)
                pass
            else:
                infra += 'Misch, '
                #print(infra)
            #------Verkehrszeichen------
            if not pd.isnull(gdf_sort['cycleway_traffic_sign'][i]):
                if row['cycleway_traffic_sign'] == 'none' or row['cycleway_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_traffic_sign']) + ', '
            if not pd.isnull(gdf_sort['cycleway_both_traffic_sign'][i]):
                if row['cycleway_both_traffic_sign'] == 'none' or row['cycleway_both_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_both_traffic_sign']) + ', '
            if not pd.isnull(gdf_sort['cycleway_left_traffic_sign'][i]):     #1xggrichtung
                if row['cycleway_left_traffic_sign'] == 'none' or row['cycleway_left_traffic_sign'] == 'None':
                    infra+= 'VZ:kein_VZ, '
                else:
                    infra+= 'VZ:' + str(row['cycleway_left_traffic_sign']) + ', '
            #------marking--------------
            if row['cycleway_left_marking_left']=='solid_line':    #4xggrichtung
                infra+='linkeM_SL, '
            elif row['cycleway_left_marking_left']=='double_solid_line':
                infra+='linkeM_DSL, '
            elif row['cycleway_left_marking_left']=='dashed_line':
                infra+='linkeM_DL, '
            elif row['cycleway_left_marking_left']=='barred_area':
                infra+='linkeM_BA, '
            if row['cycleway_left_marking_right']=='solid_line':   #4xggrichtung
                infra+='rechteM_SL,'
            elif row['cycleway_left_marking_right']=='double_solid_line':
                infra+='rechteM_DSL, '
            elif row['cycleway_left_marking_right']=='dashed_line':
                infra+='rechteM_DL, '
            elif row['cycleway_left_marking_right']=='barred_area':
                infra+='rechteM_BA, '
            if row['cycleway_left_marking_both']=='solid_line':     #4xggrichtung
                infra+='rechteM_SL, linkeM_SL, '
            elif row['cycleway_left_marking_both']=='double_solid_line':
                infra+='rechteM_DSL, linkeM_DSL, '
            elif row['cycleway_left_marking_both']=='dashed_line':
                infra+='rechteM_DL, linkeM_DL, '
            elif row['cycleway_left_marking_both']=='barred_area':
                infra+='rechteM_BA, linkeM_BA, '
            if row['cycleway_both_marking_left']=='solid_line':
                infra+='linkeM_SL, '
            elif row['cycleway_both_marking_left']=='double_solid_line':
                infra+='linkeM_DSL, '
            elif row['cycleway_both_marking_left']=='dashed_line':
                infra+='linkeM_DL, '
            elif row['cycleway_both_marking_left']=='barred_area':
                infra+='linkeM_BA, '
            if row['cycleway_both_marking_right']=='solid_line':
                infra+='rechteM_SL, '
            elif row['cycleway_both_marking_right']=='double_solid_line':
                infra+='rechteM_DSL, '
            elif row['cycleway_both_marking_right']=='dashed_line':
                infra+='rechteM_DL, '
            elif row['cycleway_both_marking_right']=='barred_area':
                infra+='rechteM_BA, '
            if row['cycleway_both_marking_both']=='solid_line':
                infra+='rechteM_SL, linkeM_SL, '
            elif row['cycleway_both_marking_both']=='double_solid_line':
                infra+='rechteM_DSL, linkeM_DSL, '
            elif row['cycleway_both_marking_both']=='dashed_line':
                infra+='rechteM_DL, linkeM_DL, '
            elif row['cycleway_both_marking_both']=='barred_area':
                infra+='rechteM_BA, linkeM_BA, '
            #------Separation-----------
            if row['cycleway_left_separation_left']=='kerb':    #6xggrichtung
                infra+='linkeSep_Bord, '
            elif row['cycleway_left_separation_left']=='greenery':
                infra+='linkeSep_Gruen, '
            if row['cycleway_left_separation_right']=='kerb':
                infra+='rechteSep_Bord, '
            elif row['cycleway_left_separation_right']=='greenery':
                infra+='rechteSep_Gruen, '
            if row['cycleway_left_separation_both']=='kerb':
                infra+='linkeSep_Bord, rechteSep_Bord, '
            elif row['cycleway_left_separation_both']=='greenery':
                infra+='linkeSep_Gruen, rechteSep_Gruen, '
            if row['cycleway_both_separation_left']=='kerb':
                infra+='linkeSep_Bord, '
            elif row['cycleway_both_separation_left']=='greenery':
                infra+='linkeSep_Gruen, '
            if row['cycleway_both_separation_right']=='kerb':
                infra+='rechteSep_Bord, '
            elif row['cycleway_both_separation_right']=='greenery':
                infra+='rechteSep_Gruen, '
            if row['cycleway_both_separation_both']=='kerb':
                infra+='linkeSep_Bord, rechteSep_Bord, '
            elif row['cycleway_both_separation_both']=='greenery':
                infra+='linkeSep_Gruen, rechteSep_Gruen, '
            #------Buffer---------------
            if not pd.isnull(gdf_sort['cycleway_left_buffer_right'][i]):    #3 bzw 4x ggrichtung
                infra+='rechterPuffer:' + row['cycleway_left_buffer_right'] + ',' 
                gdf_sort['PufferRechts'][i]=row['cycleway_left_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_left_buffer_left'][i]):
                infra+='linkerPuffer:' + row['cycleway_left_buffer_left'] + ','
                gdf_sort['PufferLinks'][i]=row['cycleway_left_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_left_buffer_both'][i]):
                infra+='rechterPuffer:' + row['cycleway_left_buffer_both'] + ', linkerPuffer:' + row['cycleway_left_buffer_both'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_left_buffer_both']
                gdf_sort['PufferLinks'][i]=row['cycleway_left_buffer_both']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_right'][i]):
                infra+='rechterPuffer:' + row['cycleway_both_buffer_right'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_both_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_left'][i]):
                infra+='linkerPuffer:' + row['cycleway_both_buffer_left'] + ','
                gdf_sort['PufferLinks'][i]=row['cycleway_both_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_both'][i]):
                infra+='rechterPuffer:' + row['cycleway_both_buffer_both'] + ', linkerPuffer:' + row['cycleway_both_buffer_both'] + ','
                gdf_sort['PufferRechts'][i]=row['cycleway_both_buffer_both']
                gdf_sort['PufferLinks'][i]=row['cycleway_both_buffer_both']
            #------Buffer - Gegenrichtung------
            if not pd.isnull(gdf_sort['cycleway_right_buffer_right'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_right_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_right_buffer_left'][i]):
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_right_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_right_buffer_both'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_right_buffer_both']
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_right_buffer_both']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_right'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_both_buffer_right']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_left'][i]):
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_both_buffer_left']
            elif not pd.isnull(gdf_sort['cycleway_both_buffer_both'][i]):
                gdf_sort['GG-PufferRechts'][i]=row['cycleway_both_buffer_both']
                gdf_sort['GG-PufferLinks'][i]=row['cycleway_both_buffer_both']            
            #------Radwegbreite-----------------
            if not pd.isnull(gdf_sort['cycleway_left_width'][i]):
                infra+='Radbreite:' + row['cycleway_left_width'] + ',' #ggrichtung
                gdf_sort['Radwegbreite'][i]=row['cycleway_left_width']
            elif not pd.isnull(gdf_sort['cycleway_both_width'][i]):
                infra+='Radbreite:' + row['cycleway_both_width'] + ','
                gdf_sort['Radwegbreite'][i]=row['cycleway_both_width']
            elif not pd.isnull(gdf_sort['cycleway_width'][i]):
                infra+='Radbreite:' + row['cycleway_width'] + ','
                gdf_sort['Radwegbreite'][i]=row['cycleway_width']
            #------Radwegbreite- Gegenrichtung--
            if not pd.isnull(gdf_sort['cycleway_right_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_right_width']
            elif not pd.isnull(gdf_sort['cycleway_both_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_both_width']
            elif not pd.isnull(gdf_sort['cycleway_width'][i]):
                gdf_sort['GG-Radwegbreite'][i]=row['cycleway_width']
            #------Parken-----------------------
            if row['parking_lane_both']=='perpendicular':
                infra+='senkrechtparken,'
            elif row['parking_lane_both']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_lane_both']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_lane_both']=='seperat':
                infra+='parken_extra,'
            elif row['parking_lane_both']=='marked':
                infra+='parken_extra,'
            elif row['parking_lane_both']=='no' or row['parking_lane_both']=='none':
                infra+='kein_P,'
            if row['parking_lane_left']=='perpendicular': #5xggrichtung
                infra+='senkrechtparken,'
            elif row['parking_lane_left']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_lane_left']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_lane_left']=='seperat':
                infra+='parken_extra,'
            elif row['parking_lane_left']=='marked':
                infra+='parken_extra,'
            elif row['parking_lane_left']=='no' or row['parking_lane_left']=='none':
                infra+='kein_P,'
            if row['parking']=='lane':
                infra+='Parkstrei., '
            elif row['parking']=='street_side':
                infra+='Parkbucht, '
            elif row['parking']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking']=='seperate':
                infra+='parken_extra,'
            elif row['parking']=='marked':
                infra+='parken_extra,'
            elif row['parking']=='no' or row['parking']=='none':
                infra+='kein_P, '
            if row['parking_left']=='lane': #5xggrichtung
                infra+='Parkstrei., '
            elif row['parking_left']=='street_side':
                infra+='Parkbucht, '
            elif row['parking_left']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking_left']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking_left']=='separate':
                infra+='parken_extra, '
            elif row['parking_left']=='no' or row['parking_left']=='none':
                infra+='kein_P, '
            if row['parking_both']=='lane':
                infra+='Parkstrei., '
            elif row['parking_both']=='street_side':
                infra+='Parkbucht, '
            elif row['parking_both']=='on_kerb':
                infra+='parken_aufgesetzt, '
            elif row['parking_both']=='half_on_kerb':
                infra+='parken_halb-aufgesetzt, '
            elif row['parking_both']=='separate':
                infra+='parken_extra, '
            elif row['parking_both']=='no' or row['parking_both']=='none':
                infra+='kein_P, '
            if row['parking_orientation']=='perpendicular': 
                infra+='senkrechtparken,'
            elif row['parking_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_orientation']=='marked':
                infra+='parken_extra,'
            if row['parking_left_orientation']=='perpendicular': #5xggrichtung
                infra+='senkrechtparken,'
            elif row['parking_left_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_left_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_left_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_left_orientation']=='marked':
                infra+='parken_extra,'
            if row['parking_both_orientation']=='perpendicular': 
                infra+='senkrechtparken,'
            elif row['parking_both_orientation']=='parallel':
                infra+='laengsparken,'   
            elif row['parking_both_orientation']=='diagonal':
                infra+='schraegparken,' 
            elif row['parking_both_orientation']=='seperat': #? Sollte nicht vorkommen
                infra+='parken_extra,'
            elif row['parking_both_orientation']=='marked':
                infra+='parken_extra,'
            #-----Parken-Gegenrichtung--------
            pinfra+='(P-gg: '
            if row['parking_lane_both']=='perpendicular':
                pinfra+='senkrechtparken,'
            elif row['parking_lane_both']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_lane_both']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_lane_both']=='seperat':
                pinfra+='parken_extra,'
            elif row['parking_lane_both']=='marked':
                pinfra+='parken_extra,'
            elif row['parking_lane_both']=='no' or row['parking_lane_both']=='none':
                pinfra+='kein_P,'
            if row['parking_lane_right']=='perpendicular': #5xggrichtung
                pinfra+='senkrechtparken,'
            elif row['parking_lane_right']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_lane_right']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_lane_right']=='seperat':
                pinfra+='parken_extra,'
            elif row['parking_lane_right']=='marked':
                pinfra+='parken_extra,'
            elif row['parking_lane_right']=='no' or row['parking_lane_right']=='none':
                pinfra+='kein_P,'
            if row['parking']=='lane':
                pinfra+='Parkstrei., '
            elif row['parking']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking']=='no' or row['parking']=='none':
                pinfra+='kein_P, '
            if row['parking_right']=='lane': #5xggrichtung
                pinfra+='Parkstrei., '
            elif row['parking_right']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking_right']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking_right']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking_right']=='separate':
                pinfra+='parken_extra, '
            elif row['parking_right']=='no' or row['parking_right']=='none':
                pinfra+='kein_P, '
            if row['parking_both']=='lane':
                pinfra+='Parkstrei., '
            elif row['parking_both']=='street_side':
                pinfra+='Parkbucht, '
            elif row['parking_both']=='on_kerb':
                pinfra+='parken_aufgesetzt, '
            elif row['parking_both']=='half_on_kerb':
                pinfra+='parken_halb-aufgesetzt, '
            elif row['parking_both']=='separate':
                pinfra+='parken_extra, '
            elif row['parking_both']=='no' or row['parking_both']=='no':
                pinfra+='kein_P, '
            if row['parking_orientation']=='perpendicular': 
                pinfra+='senkrechtparken,'
            elif row['parking_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_orientation']=='marked':
                pinfra+='parken_extra,'
            if row['parking_right_orientation']=='perpendicular': #5xggrichtung
                pinfra+='senkrechtparken,'
            elif row['parking_right_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_right_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_right_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_right_orientation']=='marked':
                pinfra+='parken_extra,'
            if row['parking_both_orientation']=='perpendicular': 
                pinfra+='senkrechtparken,'
            elif row['parking_both_orientation']=='parallel':
                pinfra+='laengsparken,'   
            elif row['parking_both_orientation']=='diagonal':
                pinfra+='schraegparken,' 
            elif row['parking_both_orientation']=='seperat': #? Sollte nicht vorkommen
                pinfra+='parken_extra,'
            elif row['parking_both_orientation']=='marked':
                pinfra+='parken_extra,'
            pinfra+='), '
            if pinfra == '(P-gg: ), ':
                pinfra=''
            infra += pinfra
            #-----Straßenbreite---------------
            if not pd.isnull(gdf_sort['width'][i]):
                infra+='Gesamtbreite:' + row['width'] + ','
            #-----Einbahnstr. oder nicht?-----
            if row['oneway']=='yes':
                infra+='Einricht,'
            elif row['oneway']=='no':
                infra+='Zweiricht,' 

           # if infra=='-1,':
           #     infra+='Misch'
                        
            
    if infra != '':
        gdf_sort.at[i, 'Radinfra'] = infra   #an dieser Stelle wird Warnung 'A value is trying to be set on a copy of a slice from a DataFrame.' verursacht. Funzt aber..
    else:
        gdf_sort.at[i, 'Radinfra'] = 'unbekannt'

# ------------ Berechnung der Breite der Kernfahrbahn (=Fahrbahn - Parken - Radstreifen - Puffer) -------------

q=0    #Nur um zu zählen, wie oft das vor kommt.
q2=0

#Funktion zum Berechnen der Parkplatzbreiten
def parken_abziehen(pinfo, breit):
    abzug = 0
    
    if 'laengs' in pinfo:
        abzug = 2    #Annahme: Längsparken am Hochbord nach EAR 2005 (Tabelle F-1, Seite 91)
    elif 'schraeg' in pinfo:
        abzug = 4.6  #Annahme: Schrägparken mit 70gon am Hochbord nach EAR 2005 (Tabelle F-1, Seite 91)
    elif 'senkrecht' in pinfo:
        abzug = 4.3  #Annahme: Senkrechtparken am Hochbord nach EAR 2005 (Tabelle F-1, Seite 91)
    if 'parken_halb-aufgesetzt' in pinfo:
        abzug = abzug / 2  #Annahme: Bei halb aufgesetztem Parken steht nur die Hälfte auf der Straße.
    elif 'Parkstrei.' in pinfo:
        abzug = abzug

    breit = breit - abzug  
    #print(str(breit) + ' ' + pinfo + ':, ' + str(abzug))
    return(breit)

#-- Gesucht wird nach allen Parkstrei. und Fällen von halb aufgesetztem Parken. ParkParkbuchten und ganz aufgesetztes sollten keinen Einfluss auf die Breite haben.
#-- Anschließend werden die Breiten der Radstreifen und Puffer abgezogen. Breiten von Fahrbahnmarkierungen werden nicht berücksichtigt.
for i,row in gdf_sort.iterrows():
    gesbreite=''    #Gesamtbreite vor Abzügen
    fliessbreite='' #Breite ohne Parken
    kern=''         #Breite ohne Parken, Radwege und Puffer
    
    if not pd.isna(row['width']):           #nur machen, wenn es zu der Straße überhaupt eine Breitenangabe gibt.       
        gesbreite = float(row['width'])
        if 'park' in str(row['Radinfra']): #gibt es an dem Way parken?
            #print(str(row['Radinfra']))
            try:
                #such auf der Seite des Überholabstands (Vor den gesuchten Strings kommt KEIN 'P-gg')
                #r           : es folgt regex. In '' der regexausdruck
                #^           : fang vorne an
                #.*          : jedes Beliebige Zeichen kann 0 bis ganz oft kommen
                #(?<!P-gg: ) : Wenn du was gefunden hast, prüfe, dass davor NICHT (!) der Ausdruck 'P-gg ' steht
                #(xxx.*?X|xxx.*?x)
                # |          : logisches ODER (das davor ODER das danach soll gefunden werden)
                # xx,.*?,    : sucht xx, gefolgt von so wenig Zeichen wie möglich bis zum nächsten ','
                #  dabei: .* : jedes Beliebiege Zeichen 0 bis ganz oft
                #  und     ? : aber nur bis zum nächsten ','
                
                p1 = re.search(r'^.*(?<!P-gg: )(parken_halb-aufgesetzt,.*?,|Parkstrei.,.*?,)', str(row['Radinfra']))
                if p1 is not None:
                    q=q+1
                    fliessbreite = parken_abziehen(p1.group(1), gesbreite)  #geht in Funktion und gibt die gefundenen Infos sowie die Breite der Straße mit.
                    #print(p1.group(1) + ' mit gg ' + str(i))
                #such auf der gegenüberliegenden Seite des Überholabstands (Vor den gesuchten Strings steht 'P-gg')
                p2 = re.search(r'^.*(?<=P-gg: )(parken_halb-aufgesetzt,.*?,|Parkstrei.,.*?,)', str(row['Radinfra']))
                if p2 is not None:
                    if fliessbreite == '':       #falls Parken nur in Gegenrichtung ist fliessbreite hier noch leer.
                        fliessbreite = gesbreite
                    q2=q2+1
                    fliessbreite = parken_abziehen(p2.group(1), fliessbreite)  #geht in Funktion und gibt die gefundenen Infos sowie die (Rest)Breite der Straße mit.
                    #print(p2.group(1) + ' mit gg ' + str(i))
            except BaseException as e:
                print('e1:' + str(e))
                pass
        if fliessbreite == '':
            fliessbreite = gesbreite
        
        #Breite der Fahrbahn setzen, bei Parken abzgl. Parkbreiten .            
        gdf_sort.at[i, 'Fahrbahn_Fliessverkehr'] = fliessbreite   #an dieser Stelle wird Warnung 'A value is trying to be set on a copy of a slice from a DataFrame.' verursacht. Funzt aber..
        
        #Checken, ob Radstreifenbreiten und Pufferbreiten abgezogen werden müssen
        abzug_Rad = 0
        if re.search(r'|'.join(liste_Misch), row['Radinfra']):  #liste_Misch enthält alle Werte, die Ausschlusskriterien für Mischverkehr sind
            if not pd.isna(row['Radwegbreite']) and is_number(row['Radwegbreite']):
                abzug_Rad = abzug_Rad + float(row['Radwegbreite'])
            if not pd.isna(row['PufferLinks']) and is_number(row['PufferLinks']):
                abzug_Rad = abzug_Rad + float(row['PufferLinks'])
            if not pd.isna(row['PufferRechts']) and is_number(row['PufferRechts']):  
                abzug_Rad = abzug_Rad + float(row['PufferRechts'])    
        if not pd.isna(row['GG-Radstreifen']):
            if re.search(r'|'.join(liste_Misch), row['GG-Radstreifen']):
                if not pd.isna(row['GG-Radwegbreite']) and is_number(row['GG-Radwegbreite']):
                    abzug_Rad = abzug_Rad + float(row['GG-Radwegbreite'])
                if not pd.isna(row['GG-PufferLinks']) and is_number(row['GG-PufferLinks']):
                    abzug_Rad = abzug_Rad + float(row['GG-PufferLinks'])
                if not pd.isna(row['GG-PufferRechts']) and is_number(row['GG-PufferRechts']):
                    abzug_Rad = abzug_Rad + float(row['GG-PufferRechts'])
        if 'Fahrradstr' in row['Radinfra']:    #Im falle von Puffer in Fahrradstr. wird angenommen, dass der auf beiden Seiten gleich ist.
            if not pd.isna(row['PufferRechts']) and is_number(row['PufferRechts']):
                abzug_Rad = abzug_Rad + (2 * float(row['PufferRechts']))   
           
        kern = fliessbreite - abzug_Rad  #wenn es kein Parken und/oder Radstreifen gibt ist es gleich der Gesamtbreite
        #print(gesbreite, fliessbreite, kern, abzug_Rad) 
        gdf_sort.at[i, 'Kernfahrbahn'] = kern   #an dieser Stelle wird Warnung 'A value is trying to be set on a copy of a slice from a DataFrame.' verursacht. Funzt aber..
        
        #Berechnung von Fahrstreifenbreiten falls es mehr als einen fahrstreifen gibt.
        if row['lanes'] is not None: 
            gdf_sort.at[i, 'Fahrstreifenbreite'] = kern / int(row['lanes'])   #an dieser Stelle wird Warnung 'A value is trying to be set on a copy of a slice from a DataFrame.' verursacht. Funzt aber..
            #Berechnung von Fahrstreifenbreite + Radinfra bei straßenseitiger Radinfra
            if not pd.isna(gdf_sort['Fahrstreifenbreite'][i]) and not pd.isna(row['Radwegbreite']) and re.search(r'|'.join(liste_Misch), row['Radinfra']):
                puf_this = 0
                if is_number(row['PufferLinks']) and not pd.isna(row['PufferLinks']):   #is_number: True wenn: Zahl oder nan, kein String
                    puf_this = float(row['PufferLinks'])
                if is_number(row['PufferRechts']) and not pd.isna(row['PufferRechts']):
                    puf_this = puf_this + float(row['PufferRechts'])
                gdf_sort.at[i, 'FSBundRad'] = float(gdf_sort['Fahrstreifenbreite'][i]) + float(row['Radwegbreite']) + puf_this
            if pd.isna(gdf_sort['FSBundRad'][i]):
                gdf_sort.at[i, 'FSBundRad'] = float(gdf_sort['Fahrstreifenbreite'][i])


#print(q)  #Anzahl OBS-Messwerte mit Parken
#print(q2) #Anzahl für Gegenrichtung

#setzen der Zeit zu Mitteleuropa
gdf_sort['datetime_berlin'] = gdf_sort['time'].dt.tz_convert('Europe/Berlin')  #!!!!! Geht bei export als gpkg anscheinend wieder verloren !!!!!
#Spalten mit nur Zeit bzw. nur Datum
gdf_sort['time_berlin'] = pd.to_datetime(gdf_sort['datetime_berlin']).dt.strftime('%H:%M')
gdf_sort['date_berlin'] = pd.to_datetime(gdf_sort['datetime_berlin']).dt.strftime('%d.%m.%Y')

#------ Setzen von bestimmten Spalten als richtige Datenart ----------
gdf_sort['Radwegbreite'] = gdf_sort['Radwegbreite'].astype('float')
gdf_sort['GG-Radwegbreite'] = gdf_sort['GG-Radwegbreite'].astype('float')

gdf_sort['Fahrstreifenbreite'] = gdf_sort['Fahrstreifenbreite'].astype('float')
gdf_sort['FSBundRad'] = gdf_sort['FSBundRad'].astype('float')
gdf_sort['Kernfahrbahn'] = gdf_sort['Kernfahrbahn'].astype('float')
gdf_sort['Fahrbahn_Fliessverkehr'] = gdf_sort['Fahrbahn_Fliessverkehr'].astype('float')


gdf_sort.head()

/home/lukas/anaconda3/envs/MA-working/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lukas/anaconda3/envs/MA-working/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/lukas/anaconda3/envs/MA-working/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy

,distance_overtaker,distance_stationary,direction,way_id,course,speed,time,portal,osm_id,name,highway,ref,maxspeed,zone_traffic,lanes,lane_markings,lit,surface,bicycle,cycleway,cycleway_both,cycleway_left,cycleway_right,width,tracktype,smoothness,oneway,cycleway_lane,cycleway_both_lane,cycleway_right_lane,cycleway_left_lane,parking_lane_both,parking_lane_left,parking_lane_right,cycleway_separation_left,cycleway_separation_right,separation_left,separation_right,cycleway_bicycle,cycleway_both_bicycle,...,cycleway_both_marking_left,cycleway_both_marking_right,cycleway_both_traffic_sign,cycleway_both_marking_both,cycleway_right_buffer_right,cycleway_left_buffer_right,cycleway_both_buffer_right,cycleway_right_buffer_left,cycleway_left_buffer_left,cycleway_both_buffer_left,cycleway_right_buffer_both,cycleway_left_buffer_both,cycleway_both_buffer_both,cycleway_marking_both,cycleway_marking_left,cycleway_marking,cycleway_right_marking,cycleway_left_marking,cycleway_traffic_sign,parking_orientation,other_tags,geometry_y,geometry,Radinfra,Radwegbreite,PufferRechts,PufferLinks,GG-Radstreifen,GG-Radwegbreite,GG-PufferRechts,GG-PufferLinks,Fahrstreifenbreite,FSBundRad,Kernfahrbahn,Fahrbahn_Fliessverkehr,Ortslage,Geschwindigkeit_Numerisch,datetime_berlin,time_berlin,date_berlin
0,NaN,2.64,1,141561211,2.956398,3.083333,2022-08-11 09:38:31+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32561 51.76605),"1,Radfahrstreifen,",NaN,NaN,NaN,"1,Radfahrstreifen,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,inner,50,2022-08-11 11:38:31+02:00,11:38,11.08.2022
1,1.67,1.70,1,141561211,2.895165,3.750000,2022-08-11 09:38:32+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32553 51.76606),"1,Radfahrstreifen,",NaN,NaN,NaN,"1,Radfahrstreifen,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,inner,50,2022-08-11 11:38:32+02:00,11:38,11.08.2022
2,0.85,2.79,1,141561211,2.849412,4.055556,2022-08-11 09:38:34+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32541 51.76607),"1,Radfahrstreifen,",NaN,NaN,NaN,"1,Radfahrstreifen,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,inner,50,2022-08-11 11:38:34+02:00,11:38,11.08.2022
3,1.26,NaN,1,141561211,2.900393,4.250000,2022-08-11 09:38:36+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,None,None,yes,asphalt,designated,None,lane,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"""maxspeed:conditional""=>""30 @ (Mo-Fr 07:00-17:...","LINESTRING (14.3269026 51.7658807, 14.3268727 ...",POINT (14.32527 51.76609),"1,Radfahrstreifen,",NaN,NaN,NaN,"1,Radfahrstreifen,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,inner,50,2022-08-11 11:38:36+02:00,11:38,11.08.2022
4,1.96,2.12,1,141561211,2.910295,4.305556,2022-08-11 09:38:37+00:00,adfc-brandenburg,141561211,Universitätsstraße,tertiary,None,50,None,N

In [6]:
df_Verteilung=gdf_sort['FSBundRad'].count()
#df_Verteilung=gdf_sort.groupby(['FSBundRad','cycleway_left','direction']).count()#.nlargest(60,'way_id') 
#df_Verteilung=gdf_sort[gdf_sort['Radinfra'].str.contains('Misch')].groupby(['Fahrstreifenbreite']).count()#.nlargest(60,'distance_overtaker') 

#df_Verteilung=gdf_sort.groupby(['Ortslage','Geschwindigkeit_Numerisch' ]).count().nlargest(60,'way_id') 
#df_Verteilung=gdf_sort[gdf_sort['Radinfra'].str.contains('Busf')].groupby(['Radinfra' ]).count().nlargest(60,'way_id') 
df_Verteilung

494

In [7]:
gdf_kombi = gdf_sort
datum=date.today().strftime("%y%m%d")

gdf_kombi = gpd.GeoDataFrame(gdf_kombi, geometry='geometry', crs='EPSG:4326')
gdf_kombi = gdf_kombi.astype({'geometry_y':'string'})
#gdf_kombi.to_file("../OSM-OBS_klass/OSM-OBS_Klass_"+datum+".gpkg", driver="GPKG")
#gdf_kombi.to_file("../OSM-OBS_klass/OSM-OBS_Klass_230315.gpkg", driver="GPKG")

In [ ]:


#---ab hier nur noch zum Testen verwendeter Restcode ---



In [25]:
#gdf_sort.groupby(gdf_sort['cycleway_traffic_sign'])
pd.unique(gdf_sort['cycleway_both_traffic_sign'])

array([nan, 'DE:237', 'none', 'de:237'], dtype=object)

In [23]:
pd.unique(gdf_sort['cycleway_right_traffic_sign'])

array([nan, 'de:237', 'none', 'DE:237', 'DE:340', 'DE:237,DE:1026-32',
       'de:241'], dtype=object)

In [12]:
pd.unique(gdf_sort['cycleway_left_traffic_sign'])

array([nan, 'none', 'DE:237', 'DE:241', 'de:240', 'DE:240'], dtype=object)

In [13]:
pd.unique(gdf_sort['cycleway_traffic_sign'])

array([nan, 'none', 'DE:340'], dtype=object)

In [27]:
gdf_sort['cycleway_both_traffic_sign'].value_counts()

cycleway_both_traffic_sign
none      2354
DE:237     372
de:237       4
Name: count, dtype: int64

In [31]:
gdf_test = gdf_sort[gdf_sort['cycleway_both_traffic_sign'] == 'none']
gdf_test.count()

distance_overtaker        2328
distance_stationary        883
direction                 2354
way_id                    2354
course                    2354
                          ... 
Kernfahrbahn              2227
Fahrbahn_Fliessverkehr    2227
datetime_berlin           2354
time_berlin               2354
date_berlin               2354
Length: 106, dtype: int64